## 155: Implementación de roles de usuario con JWT en FastAPI

Agregar soporte para roles de usuario (por ejemplo: admin, user, moderator) en el sistema de autenticación con JWT, y usar esos roles para restringir el acceso a ciertas rutas.

🧩 Código completo: JWT con roles y rutas basadas en rol
python
Copiar
Editar
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer
from jose import jwt, JWTError
from typing import Optional
from datetime import datetime, timedelta

app = FastAPI()

# Configuración
SECRET_KEY = "CLAVE_SUPER_SECRETA"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="login")

# Función para crear token con rol
def crear_token(email: str, rol: str) -> str:
    expiracion = datetime.utcnow() + timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    datos = {"sub": email, "rol": rol, "exp": expiracion}
    token = jwt.encode(datos, SECRET_KEY, algorithm=ALGORITHM)
    return token

# Función para obtener datos del usuario desde el token
def obtener_usuario_actual(token: str = Depends(oauth2_scheme)):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        email: str = payload.get("sub")
        rol: str = payload.get("rol")
        if email is None or rol is None:
            raise HTTPException(status_code=401, detail="Token inválido")
        return {"email": email, "rol": rol}
    except JWTError:
        raise HTTPException(status_code=401, detail="Token inválido o expirado")

# Ruta para crear token (simula login)
@app.post("/login")
def login(email: str, rol: str):
    # ⚠️ En un sistema real, aquí validarías las credenciales
    token = crear_token(email, rol)
    return {"access_token": token, "token_type": "bearer"}

# Ruta accesible solo para usuarios admin
@app.get("/solo-admin")
def solo_admin(usuario=Depends(obtener_usuario_actual)):
    if usuario["rol"] != "admin":
        raise HTTPException(status_code=403, detail="Acceso restringido solo para administradores")
    return {"mensaje": f"Bienvenido administrador {usuario['email']}"}

# Ruta accesible para cualquier usuario autenticado
@app.get("/perfil")
def perfil(usuario=Depends(obtener_usuario_actual)):
    return {"email": usuario["email"], "rol": usuario["rol"]}